# MSCI 436 Team 8 ML Decision Model
1. Import packages
2. Load Data
3. Clean Data
4. Fit a LR model
5. Model Evaluation
6. Streamlit
7. References for code

# Import Packages

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Initial Model - All feature variables selected

**Load Data**

In [ ]:
# Loading in required data
raw_data_train = pd.read_csv('https://raw.githubusercontent.com/jmpark0808/pl_mnist_example/main/train_hp_msci436.csv')
raw_data_train.head()


In [ ]:
raw_data_train.tail()


In [ ]:
# View shape of data
print(raw_data_train.shape)

**Cleaning Data**

In [ ]:
df = raw_data_train
df.info()

In [ ]:
# View SalePrice metrics
df['SalePrice'].describe()

In [ ]:
# Plotting SalePrice
sns.displot(df['SalePrice']);

In [ ]:
# Scatterplot for given features
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'TotalBsmtSF', 'YearBuilt']
sns.pairplot(df[cols], height = 2)
plt.show();

**Checking Null Values**

In [ ]:
# checking for null values
print(df.isnull().sum())

In [ ]:
print(df[df['LotFrontage'].isnull() | (df['LotFrontage'] == 0)])

In this case 0 could just mean that there is no street connected to the property, 0 is fine

In [ ]:
# Citation at end of notebook
# Checking which features are the more irrelevant (most houses don't have this feature)
# Count NaN and numeric 0 values
total = df.isna().sum().sort_values(ascending=False)

# Calculate percentage of NaN and numeric 0 values
percent = (df.isna().sum() / len(df)).sort_values(ascending=False)

missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)


In [ ]:
# Dropping features with missing data
df = df.drop((missing_data[missing_data['Total'] > 1]).index,1)
df = df.drop(df.loc[df['Electrical'].isnull()].index)
df.isnull().sum().max() #just checking that there's no missing data missing...

In [ ]:
print(df.shape)
df.head()

**Checking Categorical Variables**

In [ ]:
# Get column names with 'object' data type
object_columns = df.select_dtypes(include='object').columns

# Iterate over the 'object' columns
for column in object_columns:
    unique_values = df[column].unique()
    print(f"Column: {column}")
    print(unique_values)
    print("--------------------")

In this case, 0 just means that the house doesn't have that particular feature. It's not necessarily a missing value. This is important information. For example, a user might want to know if a pool vs no pool would make a significant difference in value in case they have a budget and space to add a pool.

**One-hot encoding**

In [ ]:
# Code for reference
# Select the categorical columns to be one-hot encoded (excluding the '0' values)
categorical_columns = df.select_dtypes(include='object').columns

# Create a new DataFrame with the original '0' values intact
encoded_df = pd.get_dummies(df, columns=categorical_columns, drop_first=False)

# Print the encoded DataFrame
print(encoded_df)

In [ ]:
df = encoded_df
print(df.shape)
df.head()

**Model**

In [ ]:
# Filter for float64 and int64
df_clean = df.select_dtypes(include = ['float64', 'int64']).fillna(0)
X = df.loc[:, ~df.columns.isin(['SalePrice', 'Id'])]
y = df["SalePrice"]

**Train Test Split**

In [ ]:
# Initializing the testing and training split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.70, random_state=1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

**Fit a Linear Regression Model**

In [ ]:
# Fitting the regression model and displaying the coefficient weights
reg = LinearRegression().fit(X_train, Y_train)
reg.coef_

**Model Evaluation**

In [ ]:
# MSE and MAE calculation
reg_pred = reg.predict(X_test)
reg_mse = mean_squared_error(Y_test, reg_pred)
reg_mae = mean_absolute_error(Y_test, reg_pred)

print("Mean Squared Error (MSE): ", reg_mse)
print("Mean Absolute Error (MAE): ", reg_mae)

# Revised Model With Selected Columns - Interface




In [ ]:
df_interface = raw_data_train
df_interface

In [ ]:
# Heatmap
# Correlation matrix - checking which columns strongly correlate with the sale price
corrmat = df_interface.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
# Selecting the highly correlated columns and making a new df
df_interface_v2 = df_interface[['OverallQual', 'LotFrontage','YearBuilt', 'YearRemodAdd',
                             'MasVnrArea', 'TotalBsmtSF', 'BsmtFinSF1', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath',
                             'HalfBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'SalePrice']].copy()
df_interface_v2

In [ ]:
# Checking which features are the more irrelevant (most houses don't have this feature)
# Count NaN and numeric 0 values
total = df_interface_v2.isna().sum().sort_values(ascending=False)

# Calculate percentage of NaN and numeric 0 values
percent = (df_interface_v2.isna().sum() / len(df)).sort_values(ascending=False)

# Group and print missing data
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
df_interface_v2 = df_interface_v2.drop((missing_data[missing_data['Total'] > 1]).index,1)

# Checking that there's no missing data missing...
df_interface_v2.isnull().sum().max()

**Hot encoding**

In [ ]:
# Select the categorical columns to be one-hot encoded (excluding the '0' values)
categorical_columns = df_interface_v2.select_dtypes(include='object').columns

# Create a new DataFrame with the original '0' values intact
encoded_df = pd.get_dummies(df_interface_v2, columns=categorical_columns, drop_first=False)

# Print the encoded DataFrame
print(encoded_df)

**Train & Test Data Split**

In [ ]:
# when doing experiment you want to have a controlled chaos, random_state = 1, 1 is a non-negative integer seed
# a seed will generate the same set of random numbers every time
df_clean = encoded_df.select_dtypes(include = ['float64', 'int64']).fillna(0)
X = encoded_df.loc[:, ~encoded_df.columns.isin(['SalePrice', 'Id'])]
y = encoded_df["SalePrice"]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

In [ ]:
X_test.columns

**Linear Regression**

In [ ]:
# Fitting LR and displaying coefficents
reg = LinearRegression().fit(X_train, Y_train)
reg.coef_

In [ ]:
# predict values
y_pred = reg.predict(X_test)

**Model Evaluation**

In [ ]:
# MSE and MAE calculations
reg_pred = reg.predict(X_test)
reg_mse = mean_squared_error(Y_test, reg_pred)
reg_mae = mean_absolute_error(Y_test, reg_pred)

print("Mean Squared Error (MSE): ", reg_mse)
print("Mean Absolute Error (MAE): ", reg_mae)

**Dataframe with Predicted Data**

In [ ]:
merged_data = np.concatenate((X_test, reg_pred.reshape(-1, 1)), axis=1)
column_names = list(X_test.columns) + ['SalePrice']
df_merged = pd.DataFrame(merged_data, columns=column_names)
df_merged


# Streamlit

In [ ]:
# Required package installations
!pip install -q streamlit
!npm install localtunnel

In [ ]:
df_merged.to_csv('out.csv', index=False)

In [ ]:
%%writefile app.py

from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_any_dtype,
    is_numeric_dtype,
    is_object_dtype,
)
import pandas as pd
import streamlit as st
import numpy as np
import altair as alt
import plotly.graph_objects as go

st.title("ML-driven House Customization for Maximum Profit")
st.caption("MSCI 436 - Team 8")

#filtering
st.header("Filtered Data")
def filter_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    st.caption("To filter through the columns, select the columns you would like.")
    modify = st.checkbox("Add filters")

    if not modify:
        return df

    df = df.copy()

    # Try to convert datetimes into a standard format (datetime, no timezone)
    for col in df.columns:
        if is_object_dtype(df[col]):
            try:
                df[col] = pd.to_datetime(df[col])
            except Exception:
                pass

        if is_datetime64_any_dtype(df[col]):
            df[col] = df[col].dt.tz_localize(None)

    modification_container = st.container()

    with modification_container:
        to_filter_columns = st.multiselect("Filter dataframe on", df.columns)
        for column in to_filter_columns:
            left, right = st.columns((1, 20))
            # Treat columns with < 10 unique values as categorical
            if is_categorical_dtype(df[column]) or df[column].nunique() < 10:
                user_cat_input = right.multiselect(
                    f"Values for {column}",
                    df[column].unique(),
                    default=list(df[column].unique()),
                )
                df = df[df[column].isin(user_cat_input)]
            elif is_numeric_dtype(df[column]):
                _min = float(df[column].min())
                _max = float(df[column].max())
                step = (_max - _min) / 100
                user_num_input = right.slider(
                    f"Values for {column}",
                    min_value=_min,
                    max_value=_max,
                    value=(_min, _max),
                    step=step,
                )
                df = df[df[column].between(*user_num_input)]
            elif is_datetime64_any_dtype(df[column]):
                user_date_input = right.date_input(
                    f"Values for {column}",
                    value=(
                        df[column].min(),
                        df[column].max(),
                    ),
                )
                if len(user_date_input) == 2:
                    user_date_input = tuple(map(pd.to_datetime, user_date_input))
                    start_date, end_date = user_date_input
                    df = df.loc[df[column].between(start_date, end_date)]
            else:
                user_text_input = right.text_input(
                    f"Substring or regex in {column}",
                )
                if user_text_input:
                    df = df[df[column].astype(str).str.contains(user_text_input)]

    return df

df_ui = pd.read_csv('out.csv')
st.dataframe(filter_dataframe(df_ui))

#Graphs
st.header("Visualization")

#all features
st.subheader("All features")
st.caption("Showing the relationship between all the features and the house price.")
cols = df_ui.columns.tolist()[:-1]
st.line_chart(df_ui, x = "SalePrice", y = cols, )

#line graph
def line_graph():
  st.subheader("Line Graphs: Quality, Year, Area")
  st.caption("Showing the relationship between features of the house and the sale price.")
  tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8 = st.tabs(["Overall Quality", "Year Built",
                      "Year Remodel", "Total Basement SF", "Above Ground Sqft",
                      "Garage Area", "Wood Deck SF", "Open Porch SF"])
  with tab1:
   st.line_chart(df_ui, x = "OverallQual", y = "SalePrice")
   st.text("The overall quality of the house is measured from 1-10. As shown, as the overall\n quality increases, so does the price.")
  with tab2:
    st.line_chart(df_ui, x = "YearBuilt", y = "SalePrice")
    st.text("The graph shows the relationship between the year built and the house sale price.\n Indicating that the newer house prices are higher comapred to the houses built from\n 1900s-1950s.")
  with tab3:
    st.line_chart(df_ui, x = "YearRemodAdd", y = "SalePrice")
    st.text("The remodal year shows a position correlation towards the house sale price. As the\n remodal year becomes recent, the house sale price increases as well.")
  with tab4:
    st.line_chart(df_ui, x = "TotalBsmtSF", y = "SalePrice")
    st.text("As shown in the graph, the average total basement surface area is between\n 600-1700 sqft, with the average house price range being approximately $100k to $300k.")
  with tab5:
    st.line_chart(df_ui, x = "GrLivArea", y = "SalePrice")
    st.text("This graph shows a strong positive correlation between the above ground sqft and\n the house sale price.")
  with tab6:
    st.line_chart(df_ui, x = "GarageArea", y = "SalePrice")
    st.text("This graph shows the average garage area is between 600-850 sqft, with the highest\n price at $440,101 with 831 sqft.")
  with tab7:
    st.line_chart(df_ui, x = "WoodDeckSF", y = "SalePrice")
    st.text("The graph shows that the average wood deck surface area is approximately between\n 50-450 sqft.")
  with tab8:
    st.line_chart(df_ui, x = "OpenPorchSF", y = "SalePrice")
    st.text("")

line_graph()

#plots with values that can be controlled
def plot():
  st.subheader("Line Graphs: Total Rooms, Baths, Garage Cars, Fireplaces")
  tab1, tab2, tab3, tab4, tab5 = st.tabs(["Total Rooms Abv Ground", "Full Bath", "Half Bath",
                                   "Garage Cars", "Fireplaces"])
  with tab1:
    st.subheader("Line Graph - Total Rooms Abv Ground vs Sale price")
    st.caption("Showcasing the relationship between the total number of rooms above the ground and the house sale price.")
    df = pd.read_csv('out.csv')
    df['TotRmsAbvGrd'] = df['TotRmsAbvGrd'].astype(str)
    clist = df["TotRmsAbvGrd"].unique().tolist()
    total_rooms = st.multiselect("Select total number of rooms", clist)
    st.text("You selected: {}".format(", ".join(total_rooms)))
    dfs = {TotRmsAbvGrd: df[df["TotRmsAbvGrd"] == TotRmsAbvGrd] for TotRmsAbvGrd in total_rooms}
    fig = go.Figure()
    for TotRmsAbvGrd, df in dfs.items():
        fig = fig.add_trace(go.Scatter(x=df["TotRmsAbvGrd"], y=df["SalePrice"], name=TotRmsAbvGrd))
    st.plotly_chart(fig)

  with tab2:
    st.subheader("Line Graph - Full Bath vs Sale price")
    st.caption("Showcasing the relationship between the number of full baths and the house sale price.")
    df = pd.read_csv('out.csv')
    df['FullBath'] = df['FullBath'].astype(str)
    clist = df["FullBath"].unique().tolist()
    full_baths = st.multiselect("Select number of full baths", clist)
    st.text("You selected: {}".format(", ".join(full_baths)))
    dfs = {FullBath: df[df["FullBath"] == FullBath] for FullBath in full_baths}
    fig = go.Figure()
    for FullBath, df in dfs.items():
        fig = fig.add_trace(go.Scatter(x=df["FullBath"], y=df["SalePrice"], name=FullBath))
    st.plotly_chart(fig)

  with tab3:
    st.subheader("Line Graph - Half Bath vs Sale price")
    st.caption("Showcasing the relationship between the number of half baths and the house sale price.")
    df = pd.read_csv('out.csv')
    df['HalfBath'] = df['HalfBath'].astype(str)
    clist = df["HalfBath"].unique().tolist()
    half_baths = st.multiselect("Select number of half baths", clist)
    st.text("You selected: {}".format(", ".join(half_baths)))
    dfs = {HalfBath: df[df["HalfBath"] == HalfBath] for HalfBath in half_baths}
    fig = go.Figure()
    for HalfBath, df in dfs.items():
        fig = fig.add_trace(go.Scatter(x=df["HalfBath"], y=df["SalePrice"], name=HalfBath))
    st.plotly_chart(fig)

  with tab4:
    st.subheader("Line Graph - Garage Cars vs Sale price")
    st.caption("Showcasing the relationship between the number of cars that can fit in the garage and the house sale price.")
    df = pd.read_csv('out.csv')
    df['Fireplaces'] = df['Fireplaces'].astype(str)
    clist = df["Fireplaces"].unique().tolist()
    garage_cars = st.multiselect("Select number of cars", clist)
    st.text("You selected: {}".format(", ".join(garage_cars)))
    dfs = {GarageCars: df[df["GarageCars"] == GarageCars] for GarageCars in garage_cars}
    fig = go.Figure()
    for GarageCars, df in dfs.items():
        fig = fig.add_trace(go.Scatter(x=df["GarageCars"], y=df["SalePrice"], name=GarageCars))
    st.plotly_chart(fig)

  with tab5:
    st.subheader("Line Graph - Fireplaces vs Sale price")
    st.caption("Showcasing the relationship between the number of fireplaces and the house sale price.")
    df = pd.read_csv('out.csv')
    df['Fireplaces'] = df['Fireplaces'].astype(str)
    clist = df["Fireplaces"].unique().tolist()
    fireplaces = st.multiselect("Select number of fireplaces", clist)
    st.text("You selected: {}".format(", ".join(fireplaces)))
    dfs = {Fireplaces: df[df["Fireplaces"] == Fireplaces] for Fireplaces in fireplaces}
    fig = go.Figure()
    for Fireplaces, df in dfs.items():
        fig = fig.add_trace(go.Scatter(x=df["Fireplaces"], y=df["SalePrice"], name=Fireplaces))
    st.plotly_chart(fig)


plot()

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!curl ipv4.icanhazip.com
!npx localtunnel --port 8501

34.74.18.89
npx: installed 22 in 3.301s
your url is: https://free-mails-ring.loca.lt


### References
1. *Auto-generate a dataframe filtering UI in Streamlit with filter_dataframe!* (2022, August 18). Streamlit. https://discuss.streamlit.io/t/auto-generate-a-dataframe-filtering-ui-in-streamlit-with-filter-dataframe/29470
2. Epogrebnyak. (n.d.). ssg-dataset/app/pages/1_🍴_Forks_and_issues.py at main · epogrebnyak/ssg-dataset. GitHub. https://github.com/epogrebnyak/ssg-dataset/blob/main/app/pages/1_%F0%9F%8D%B4_Forks_and_issues.py
3. Gusthema. (2023). *House Prices Prediction using TFDF. Kaggle.* https://www.kaggle.com/code/gusthema/house-prices-prediction-using-tfdf/notebook
4. Pmarcelino. (2022). *Comprehensive data exploration with Python. *Kaggle. https://www.kaggle.com/code/pmarcelino/comprehensive-data-exploration-with-python#4.-Missing-data
5. Richards, T. (2022). *Auto-generate a dataframe filtering UI in Streamlit with filter_dataframe! Streamlit.* https://blog.streamlit.io/auto-generate-a-dataframe-filtering-ui-in-streamlit-with-filter_dataframe/
6. st.line_chart - Streamlit Docs. (n.d.). https://docs.streamlit.io/library/api-reference/charts/st.line_chart
7. Streamlit multiselect line chart. (n.d.). Stack Overflow. https://stackoverflow.com/questions/71185462/streamlit-multiselect-line-chart
8. Valkov, V. (2021, December 9). *Predicting House Prices with Linear Regression | Machine Learning from Scratch (Part II). Medium.* https://towardsdatascience.com/predicting-house-prices-with-linear-regression-machine-learning-from-scratch-part-ii-47a0238aeac1
